In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# IMPORT RAW DATA
dataframes = []

ls = [2008, 2009, 2010, 2012, 2013, 2014, 2015, 2016]
# ls = [2014]
for year in ls:
# for year in range(2008, 2017):
    filename = f'./data/{year}.csv'
    try: 
        this_data = pd.read_csv(filename, na_values=' ')
    except:
        this_data = pd.read_csv(filename, encoding= 'unicode_escape', na_values=' ')
        # , dtype={'timestop': str}
  
    if year in range(2011, 2017):
        this_data = this_data.drop(columns=['forceuse'])
    if year in range(2013, 2017):
        this_data = this_data.rename(columns={'dettypCM': 'dettypcm', 
                                              'lineCM': 'linecm', 
                                              'detailCM': 'detailcm'})
        
    dataframes.append(this_data)

sqf_data_full = pd.concat(dataframes, ignore_index=True)
sqf_data = sqf_data_full.copy()

In [4]:
column_names = ['year', 'pistol', 'riflshot', 'asltweap', 'machgun', 'knifcuti', 'othrweap',
                'pct', 'trhsloc', 
                'ac_assoc', 'ac_cgdir', 'ac_rept', 'ac_evasv', 'ac_incid', 
                'ac_inves', 'ac_proxm', 'ac_time', 'ac_stsnd', 'ac_other',
                'cs_objcs', 'cs_descr', 'cs_casng', 'cs_lkout', 'cs_cloth', 'cs_drgtr', 
                'cs_furtv', 'cs_vcrim', 'cs_bulge', 'cs_other', 'age', 'build', 'sex', 
                'ht_feet', 'ht_inch', 'weight', 'inout', 'radio', 'perobs', 'datestop', 'timestop']

In [5]:
sqf_data = sqf_data[column_names]

In [7]:
sqf_data = sqf_data.dropna(subset=['timestop'])

In [8]:
sqf_data['datestop'] = sqf_data['datestop'].apply(lambda x: '{0:0>8}'.format(x))
sqf_data['timestop'] = sqf_data['timestop'].apply(lambda x: '{0:0>4}'.format(x))
# sqf_data = sqf_data[~sqf_data['timestop'].str.startswith('0n')]

sqf_data['month'] = sqf_data['datestop'].str[:2].astype(int)
sqf_data['day'] = sqf_data['datestop'].str[2:4].astype(int)
sqf_data['year'] = sqf_data['year'].astype(int)

sqf_data['time_period'] = sqf_data['timestop'].str[:2].astype(int)

In [9]:
## Handle 2014

coln = ["cs_objcs", "cs_descr", "cs_casng","cs_lkout", "cs_cloth", 
        "cs_drgtr", "cs_furtv", "cs_vcrim", "cs_bulge", "cs_other",
        "ac_rept", "ac_inves", "ac_proxm", "ac_evasv", "ac_assoc", "ac_cgdir",
        "ac_incid", "ac_time", "ac_stsnd", "ac_other", "pistol", "riflshot", "asltweap", 
        "knifcuti", "machgun", "othrweap"]

# replace values for 2014 and convert to 'Y' or 'N'
for i in coln:
    sqf_data.loc[(sqf_data['year'] == 2014) & (sqf_data[i] == 1), i] = 'Y'
    sqf_data.loc[(sqf_data['year'] == 2014) & (sqf_data[i].isna()), i] = 'N'

In [10]:
# 'pistol', 'riflshot', 'asltweap', 'machgun', 'knifcuti', 'othrweap'

def recode_yn(f):
    f_new = f.replace({'N': 0, 'Y': 1})
    f_new = f_new.astype(bool)
    return f_new

sqf_data = sqf_data.assign(found_pistol = recode_yn(sqf_data['pistol']),
                           found_rifle = recode_yn(sqf_data['riflshot']),
                           found_assault = recode_yn(sqf_data['asltweap']),
                           found_machinegun = recode_yn(sqf_data['machgun']),
                           found_knife = recode_yn(sqf_data['knifcuti']),
                           found_other = recode_yn(sqf_data['othrweap']))

sqf_data['found_weapon'] = (sqf_data['found_pistol'] | sqf_data['found_rifle'] |
                            sqf_data['found_assault'] | sqf_data['found_machinegun'] |
                            sqf_data['found_knife'] | sqf_data['found_other'])

In [12]:
sqf_data = sqf_data.assign(precinct=pd.factorize(sqf_data['pct'])[0]+1)

In [13]:
# create dictionary for recoding
recode_dict = {'P': 'neither', 'H': 'housing', 'T': 'transit'}

# add new column to sqf_data
sqf_data['location.housing'] = sqf_data_full['trhsloc'].replace(recode_dict).fillna('neither')

In [14]:
sqf_data = sqf_data.assign(additional_associating = recode_yn(sqf_data['ac_assoc']),
                           additional_direction = recode_yn(sqf_data['ac_cgdir']),
                           additional_report = recode_yn(sqf_data['ac_rept']),
                           additional_evasive = recode_yn(sqf_data['ac_evasv']),
                           additional_highcrime = recode_yn(sqf_data['ac_incid']),
                           additional_investigation = recode_yn(sqf_data['ac_inves']),
                           additional_proximity = recode_yn(sqf_data['ac_proxm']),
                           additional_time = recode_yn(sqf_data['ac_time']),
                           additional_sights = recode_yn(sqf_data['ac_stsnd']),
                           additional_other = recode_yn(sqf_data['ac_other']))

In [16]:
sqf_data = sqf_data.assign(stopped_bulge = recode_yn(sqf_data['cs_objcs']),
                           stopped_object = recode_yn(sqf_data['cs_descr']),
                           stopped_casing = recode_yn(sqf_data['cs_casng']),
                           stopped_clothing = recode_yn(sqf_data['cs_lkout']),
                           stopped_desc = recode_yn(sqf_data['cs_cloth']),
                           stopped_drugs = recode_yn(sqf_data['cs_drgtr']),
                           stopped_furtive = recode_yn(sqf_data['cs_furtv']),
                           stopped_lookout = recode_yn(sqf_data['cs_vcrim']),
                           stopped_violent = recode_yn(sqf_data['cs_bulge']),
                           stopped_other = recode_yn(sqf_data['cs_other']))

In [18]:
# add new column to sqf_data
# try:
#     sqf_data['age'] = sqf_data['age'].replace('**', -1)
#     sqf_data['suspect_age'] = sqf_data['age'].fillna(-1).astype(int).replace('-1', pd.NA)
# except:
#     sqf_data['suspect_age'] = sqf_data['age'].fillna(-1).astype(int).replace('-1', pd.NA)
sqf_data = sqf_data.drop(sqf_data[sqf_data['age'] == '**'].index)
sqf_data = sqf_data.dropna(subset=['age'])
sqf_data['suspect_age'] = sqf_data['age'].astype(int)

# replace suspect.age > 100 with NA
sqf_data.loc[sqf_data['suspect_age'] > 100, 'suspect_age'] = pd.NA
sqf_data = sqf_data.dropna(subset=['suspect_age'])

In [19]:
# create dictionary for recoding
recode_dict = {'H': 'heavy', 'M': 'medium', 'T': 'thin', 'U': 'muscular', 'Z': 'unknown'}

# add new column to sqf_data
sqf_data['suspect.build'] = sqf_data['build'].replace(recode_dict)

In [20]:
# create dictionary for recoding
recode_dict = {'M': 'male', 'F': 'female'}

# add new column to sqf_data
sqf_data['suspect.sex'] = sqf_data['sex'].replace(recode_dict)

In [21]:
sqf_data['suspect_height'] = sqf_data['ht_feet'] + (sqf_data['ht_inch'] / 12)

In [22]:
# add new column to sqf_data
sqf_data['suspect_weight'] = sqf_data['weight']

# replace suspect.weight >= 700 with NA
sqf_data.loc[sqf_data['suspect_weight'] >= 700, 'suspect_weight'] = pd.NA
sqf_data = sqf_data.dropna(subset=['suspect_weight'])

In [23]:
def recode_io(f):
    f_new = f.replace({'O': 0, 'I': 1})
    f_new = f_new.astype(bool)
    return f_new

sqf_data['inside'] = recode_io(sqf_data['inout'])
sqf_data['observation_period'] = sqf_data['perobs']

In [24]:
sqf_data = sqf_data.assign(radio_run = recode_yn(sqf_data['radio']))

In [25]:
column_names = ['pistol', 'riflshot', 'asltweap', 'machgun', 'knifcuti', 'othrweap',
                'pct', 'trhsloc', 
                'ac_assoc', 'ac_cgdir', 'ac_rept', 'ac_evasv', 'ac_incid', 
                'ac_inves', 'ac_proxm', 'ac_time', 'ac_stsnd', 'ac_other',
                'cs_objcs', 'cs_descr', 'cs_casng', 'cs_lkout', 'cs_cloth', 'cs_drgtr', 
                'cs_furtv', 'cs_vcrim', 'cs_bulge', 'cs_other', 'age', 'build', 'sex', 
                'ht_feet', 'ht_inch', 'weight', 'inout', 'radio', 'perobs', 'datestop', 'timestop']

In [26]:
sqf_data = sqf_data.drop(columns=column_names)

In [27]:
tmp_col_names = ['found_pistol', 'found_rifle', 'found_assault', 
                 'found_machinegun', 'found_knife', 'found_other']

In [28]:
sqf_data = sqf_data.drop(columns=tmp_col_names)

In [29]:
sqf_data

,year,month,day,time_period,found_weapon,precinct,location.housing,additional_associating,additional_direction,additional_report,...,stopped_violent,stopped_other,suspect_age,suspect.build,suspect.sex,suspect_height,suspect_weight,inside,observation_period,radio_run
0,2008,1,1,13,False,1,neither,False,False,False,...,False,True,20.0,muscular,male,6.000000,161.0,False,20.0,False
1,2008,1,1,20,False,2,neither,False,False,False,...,False,True,25.0,heavy,male,5.583333,190.0,True,1.0,True
2,2008,1,8,10,False,3,housing,False,False,True,...,False,False,18.0,medium,male,6.000000,180.0,False,2.0,False
3,2008,1,8,19,False,4,neither,False,False,False,...,False,False,23.0,medium,male,6.083333,185.0,False,2.0,False
4,2008,1,12,21,False,5,housing,False,False,False,...,False,True,14.0,thin,male,5.750000,145.0,True,5.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528266,2016,11,26,15,False,56,neither,False,False,False,...,False,False,19.0,medium,male,5.583333,160.0,False,30.0,True
2528267,2016,12,2,18,False,56,neither,False,False,False,...,False,False,38.0,medium,male,5.916667,210.0,False,1.0,True
2528268,2016,12,2,25,False,56,neither,False,False,False,...,False,False,22.0,unknown,male,5.666667,140.0,False,5.0,False
2528269,2016,12,4,22,False,56,neither,False,False,False,...,False,False,25.0,thin,male,5.833333,185.0,False,2.0,True


In [31]:
sqf_data = pd.read_csv('./data/sqf_data.csv')